# 决策树与Sklearn

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [ ]:
# sklearn内置y有数据集，加载了一个房价的数据集
from sklearn.datasets.california_housing import fetch_california_housing
housing = fetch_california_housing()
print(housing.DESCR)

In [ ]:
housing.data.shape

In [ ]:
housing.data[0]

In [ ]:
from sklearn import tree
# 实例化树模型，设置参数，max_depth = 2，树的最大深度
dtr = tree.DecisionTreeRegressor(max_depth = 3)
# fit中，只需要传入两个数值，要使用的数据和目标
dtr.fit(housing.data[:, [6, 7]], housing.target)

## 树模型参数:

-  1.criterion  gini  or  entropy

-  2.splitter  best or random 前者是在所有特征中找最好的切分点 后者是在部分特征中（数据量大的时候）

-  3.max_features  None（所有），log2，sqrt，N  特征小于50的时候一般使用所有的

-  4.max_depth  数据少或者特征少的时候可以不管这个值，如果模型样本量多，特征也多的情况下，可以尝试限制下

-  5.min_samples_split  如果某节点的样本数少于min_samples_split，则不会继续再尝试选择最优特征来进行划分如果样本量不大，不需要管这个值。如果样本量数量级非常大，则推荐增大这个值。

-  6.min_samples_leaf  这个值限制了叶子节点最少的样本数，如果某叶子节点数目小于样本数，则会和兄弟节点一起被剪枝，如果样本量不大，不需要管这个值，大些如10W可是尝试下5

-  7.min_weight_fraction_leaf 这个值限制了叶子节点所有样本权重和的最小值，如果小于这个值，则会和兄弟节点一起被剪枝默认是0，就是不考虑权重问题。一般来说，如果我们有较多样本有缺失值，或者分类树样本的分布类别偏差很大，就会引入样本权重，这时我们就要注意这个值了。

-  8.max_leaf_nodes 通过限制最大叶子节点数，可以防止过拟合，默认是"None”，即不限制最大的叶子节点数。如果加了限制，算法会建立在最大叶子节点数内最优的决策树。如果特征不多，可以不考虑这个值，但是如果特征分成多的话，可以加以限制具体的值可以通过交叉验证得到。

-  9.class_weight 指定样本各类别的的权重，主要是为了防止训练集某些类别的样本过多导致训练的决策树过于偏向这些类别。这里可以自己指定各个样本的权重如果使用“balanced”，则算法会自己计算权重，样本量少的类别所对应的样本权重会高。

- 10.min_impurity_split 这个值限制了决策树的增长，如果某节点的不纯度(基尼系数，信息增益，均方差，绝对差)小于这个阈值则该节点不再生成子节点。即为叶子节点 。
- n_estimators:要建立树的个数

In [ ]:
#要可视化显示 首先需要安装 graphviz   http://www.graphviz.org/Download.php
dot_data = \
    tree.export_graphviz(
        # 上面创建的树模型
        dtr,
        out_file = None,
        # 设置特征的名字
        feature_names = housing.feature_names[6:8],
        filled = True,
        impurity = False,
        rounded = True
    )

In [ ]:
#pip install pydotplus
import pydotplus
# 传入上面的dot_data
graph = pydotplus.graph_from_dot_data(dot_data)
# 设置一下颜色
graph.get_nodes()[7].set_fillcolor("#FFF2DD")
from IPython.display import Image
Image(graph.create_png())

In [ ]:
graph.write_png("dtr_white_background.png")

In [ ]:
from sklearn.model_selection import train_test_split
# 还是先切分出训练集和测试集
data_train, data_test, target_train, target_test = \
    train_test_split(housing.data, housing.target, test_size = 0.1, random_state = 42)
dtr = tree.DecisionTreeRegressor(random_state = 42)
dtr.fit(data_train, target_train)
dtr.score(data_test, target_test)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor( random_state = 42)
rfr.fit(data_train, target_train)
rfr.score(data_test, target_test)

In [ ]:
# 查找参数组合GridSearchCV，模型调参利器
from sklearn.model_selection import GridSearchCV
# 简单的设置两个参数，min_samples_split样本切分个数，n_estimators
tree_param_grid = { 'min_samples_split': list((3,6,9)),'n_estimators':list((10,50,100))}
# 先传一个算法，RandomForestRegressor，param_grid参数候选项（字典格式），cv交叉验证次数
grid = GridSearchCV(RandomForestRegressor(),param_grid=tree_param_grid, cv=5)
# 设置交叉验证使用的训练集
grid.fit(data_train, target_train)
grid.cv_results_, grid.best_params_, grid.best_score_

In [ ]:
rfr = RandomForestRegressor( min_samples_split=3,n_estimators = 100,random_state = 42)
rfr.fit(data_train, target_train)
rfr.score(data_test, target_test)

In [ ]:
pd.Series(rfr.feature_importances_, index = housing.feature_names).sort_values(ascending = False)